In [5]:
import boto3
import s3fs

client = boto3.client("s3",
                        region_name = 'sa-east-1'
                     )

In [8]:
%%time
from faker import Faker 
import json
import random
import pandas as pd
from datetime import datetime

fake = Faker()

CPU times: user 18.1 ms, sys: 0 ns, total: 18.1 ms
Wall time: 16.9 ms


In [23]:
bucket_name = 'actum-dremio-test'
prefixo_s3 = ''
folder_name = 'json_files///'

In [32]:
# Generate IoT data as a list of dictionaries
num_records = 100000

iot_data = []

for _ in range(num_records):
    sensor_data = {
        "timestamp": fake.date_time_this_decade(),
        "sensor_id": fake.uuid4(),
        "temperature": round(random.uniform(10.0, 40.0), 2),
        "humidity": round(random.uniform(30.0, 70.0), 2),
        "pressure": round(random.uniform(900.0, 1100.0), 2)
    }
    iot_data.append(sensor_data)
def datetime_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
        
# Convert the list of dictionaries to a JSON string
json_string = json.dumps(iot_data, indent=2, default = datetime_serializer)

# Specify the S3 object key for the JSON file within the bucket
s3_object_key = 'json_file_entire/iot_data.json' 

# Upload the JSON string to S3
try:
    client.put_object(Body=json_string, Bucket=bucket_name, Key=s3_object_key)
    print(f'Successfully uploaded iot_data.json to S3: {s3_object_key}')
except NoCredentialsError:
    print('Credentials not available')

Successfully uploaded iot_data.json to S3: json_file_entire/iot_data.json


In [41]:
response = client.get_object(Bucket=bucket_name, Key='json_file_entire/iot_data.json')

In [37]:
response['Body'].read().decode('utf-8')

''

In [26]:
json_string = response['Body'].read().decode('utf-8')

In [27]:
import io

In [ ]:
json_file = io.StringIO(json_string)

# Now you can use pd.read_json to create a DataFrame
df = pd.read_json(json_file, lines=True)

In [ ]:
import json

# Lê o conteúdo do corpo da resposta
content = b''
for chunk in response['Body'].iter_chunks():
    content += chunk

# Decodifica o conteúdo como uma string usando utf-8
content_str = content.decode('utf-8')

# Converte a string JSON para um DataFrame do pandas
df = pd.read_json(json.loads(content_str), orient='records', lines=True)

In [31]:
df

""


In [ ]:
response = client.get